# Definição do Projeto 

Imagine que estamos trabalhando em uma empresa de telecomunicações que oferece serviços de telefonia e internet, e temos um problema: alguns de nossos clientes estão mudando. Eles não estão mais usando nossos serviços e estão indo para um provedor diferente. Gostaríamos de evitar que isso acontecesse, por isso desenvolvemos um sistema para identificar esses clientes e oferecer-lhes um incentivo para permanecer. Queremos direcioná-los com mensagens promocionais e dar-lhes um desconto. Também gostaríamos de entender por que o modelo acha que nossos clientes mudam e, para isso, precisamos ser capazes de interpretar as previsões do modelo 

## Segue o plano do projeto:

1° Primeiro, baixamos o conjunto de dados e fazemos uma preparação inicial: renomear as colunas e alterar os valores dentro das colunas para serem consistentes em todo o conjunto de dados.

2° Em seguida, dividimos os dados em treinamento, validação e teste para que possamos validar nossos modelos.

3° Como parte da análise de dados inicial, analisamos a importância do recurso para identificar quais recursos são importantes em nossos dados.

4° Transformamos variáveis ​​categóricas em variáveis ​​numéricas para que possamos usá-las no modelo.

5° Finalmente, treinamos um modelo de regressão logística.

## Conjunto de Dados 

https://www.kaggle.com/blastchar/telco-customer-churn .

De acordo com a descrição, este conjunto de dados possui as seguintes informações:

* Serviços dos clientes: telefone; várias linhas; Internet; suporte técnico e serviços extras, como segurança online, backup, proteção de dispositivos e streaming de TV

* Dados da conta: há quanto tempo são clientes, tipo de contrato, tipo de forma de pagamento

* Cobranças: quanto foi cobrado do cliente no último mês e no total

* Informações demográficas: sexo, idade e se têm dependentes ou companheiro

* Churn: sim/não, se o cliente saiu da empresa no último mês

**`Churn`** é quando os clientes param de usar os serviços de uma empresa. Assim, a previsão do churn consiste em identificar clientes que provavelmente cancelarão seus contratos em breve. Se a empresa puder fazer isso, pode oferecer descontos nesses serviços para manter os usuários.

In [1]:
# Imports 
import pandas as pd 
import numpy as np 

import seaborn as sns 
from matplotlib import pyplot as plt 
%matplotlib inline

In [2]:
# Lendo o dataset
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [3]:
# Visualizando a quantidade de linhas que ele possui: 
len(df)

7043

In [4]:
#nem todas as colunas cabem na tela então podemos transpor para visualizar melhor
df.head().T 

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


Vemos que o conjunto de dados tem algumas colunas:

* `CustomerID`: o ID do cliente
* `Gênero` Masculino Feminino
* `SeniorCitizen`: se o cliente é idoso (0/1)
* `Companheiro`: se mora com companheiro (sim/não)
* `Dependentes`: se tem dependentes (sim/não)
* `Duração`: número de meses desde o início do contrato
* `PhoneService`: se possui atendimento telefônico (sim/não)
* `MultipleLines`: se eles têm várias linhas telefônicas (sim/não/sem serviço telefônico)
* `InternetService`: o tipo de serviço de internet (sem/fibra/óptica)
* `OnlineSecurity`: se a segurança online estiver habilitada (sim/não/sem internet)
* `OnlineBackup`: se o serviço de backup online estiver ativado (sim/não/sem internet)
* `DeviceProtection`: se o serviço de proteção do dispositivo estiver ativado (sim/não/sem internet)
* `TechSupport`: se o cliente tiver suporte técnico (sim/não/sem internet)
* `StreamingTV`: se o serviço de streaming de TV estiver habilitado (sim/não/sem internet)
* `StreamingMovies`: se o serviço de streaming de filmes estiver ativado (sim/não/sem internet)
* `Contrato`: o tipo de contrato (mensal/anual/dois anos)
* `PaperlessBilling`: se o faturamento for paperless (sim/não )
* `PaymentMethod`: método de pagamento (cheque eletrônico, cheque postal, transferência bancária, cartão de crédito)
* `Mensalidades`: o valor cobrado mensalmente (numérico)
* `TotalCharges`: o valor total cobrado (numérico)
* `Churn`: se o cliente cancelou o contrato (sim/não)

Nossa variável de destino é o `churn`. 

In [5]:
# verificando se os tipos reais estão corretos 
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

`TotalCharges`. Esperamos que essa coluna seja numérica: ela contém o valor total cobrado do cliente, portanto, deve ser um número, não uma string.

**IMPORTANTE**

Fique atento aos casos em que você espera que uma coluna seja numérica, mas o Pandas diz que não é: provavelmente a coluna contém codificação especial para valores ausentes que requerem pré-processamento adicional.

In [6]:
# forçando a coluna ser numérica 
total_charges = pd.to_numeric(df.TotalCharges, errors='coerce')
# errors='coerce' serve para fazer com que a função evite dados não numericos(como espaços) substituindo-os assim por NaN

In [7]:
# confirmando que os dados realmente contêm caracteres não numéricos
df[total_charges.isnull()][['customerID', 'TotalCharges']]
# retorna as colunas costumerid e totalcharges apenas para as linhas em que o valor da total_charges é nulo

,customerID,TotalCharges
488,4472-LVYGI,
753,3115-CZMZD,
936,5709-LVOEQ,
1082,4367-NUYAO,
1340,1371-DWPAZ,
3331,7644-OMVMY,
3826,3213-VVOLG,
4380,2520-SGTTA,
5218,2923-ARZLG,
6670,4075-WKNIU,


In [8]:
# Iremos definir esses valores ausentes como zero. 
df.TotalCharges = pd.to_numeric(df.TotalCharges, errors='coerce')
df.TotalCharges = df.TotalCharges.fillna(0)

Além disso, notamos que os nomes das colunas não seguem a mesma convenção de nomenclatura. Alguns deles começam com letra minúscula, enquanto outros começam com letra maiúscula, e também há espaços nos valores.
Vamos torná-lo uniforme, colocando tudo em letras minúsculas e substituindo os espaços por sublinhados.

In [9]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
 
string_columns = list(df.dtypes[df.dtypes == 'object'].index)
 
for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

In [10]:
# Vamos analisar nossa variável de destino churn 
df.churn.head()

0     no
1     no
2    yes
3     no
4    yes
Name: churn, dtype: object

In [11]:
# Booleano com True quando o valor é 'yes' e False quando o valor é 'No'
(df.churn == 'yes').head()

0    False
1    False
2     True
3    False
4     True
Name: churn, dtype: bool

In [12]:
# então vamos transformar em 0 e 1, sendo 0 = no, 1 = yes
df.churn = (df.churn == 'yes').astype(int)

In [13]:
#Visualizando
df.churn.head()

0    0
1    0
2    1
3    0
4    1
Name: churn, dtype: int32

In [14]:
#importando scikit-learn
from sklearn.model_selection import train_test_split

In [15]:
# Utilizando
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=1)

# random_state -> é necessário para garantir que toda vez que executarmos
# esse código, o dataframe seja dividido exatamente da mesma maneira. 

A função `train_test_split`pega um quadro de dados df e cria dois novos dataframes: `df_train_full`e `df_test`. Ele faz isso embaralhando o conjunto de dados original e, em seguida, dividindo-o de forma que o conjunto de teste contenha 20% dos dados e o conjunto de treinamento contenha os 80% restantes

In [16]:
# visualizando os indices após o train_test_split
df_train_full.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
1814,5442-pptjy,male,0,yes,yes,12,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,19.70,258.35,0
5946,6261-rcvns,female,0,no,no,42,yes,no,dsl,yes,...,yes,yes,no,yes,one_year,no,credit_card_(automatic),73.90,3160.55,1
3881,2176-osjuv,male,0,yes,no,71,yes,yes,dsl,yes,...,no,yes,no,no,two_year,no,bank_transfer_(automatic),65.15,4681.75,0
2389,6161-erdgd,male,0,yes,yes,71,yes,yes,dsl,yes,...,yes,yes,yes,yes,one_year,no,electronic_check,85.45,6300.85,0
3676,2364-ufrom,male,0,no,no,30,yes,no,dsl,yes,...,no,yes,yes,no,one_year,no,electronic_check,70.40,2044.75,0


os índices (a primeira coluna) são embaralhados nos novos quadros de dados, então, em vez de números consecutivos como 0, 1, 2, ..., eles parecem aleatórios.

A função `train_test_split` divide os dados em apenas duas partes: treinar e testar. Apesar disso, ainda podemos dividir o conjunto de dados original em três partes; apenas pegamos uma parte e dividimos novamente

In [17]:
# vamos pegar o df_train_full e dividi-lo mais uma vez em treianr e validar
df_train, df_val = train_test_split(df_train_full, test_size=0.33, random_state=11)
 
y_train = df_train.churn.values
y_val = df_val.churn.values
 
del df_train['churn']
del df_val['churn']

Agora os dataframes estão preparados e estamos prontos para usar o conjunto de dados de treinamento para realizar a análise exploratória inicial dos dados.

## Análise exploratória de dados

Devemos sempre verificar se há valores ausentes no conjunto de dados porque muitos modelos de aprendizado de máquina não podem lidar facilmente com dados ausentes. Já encontramos um problema com a coluna TotalCharges e substituímos os valores ausentes por zeros. Agora vamos ver se precisamos executar algum tratamento nulo adicional:

In [18]:
df_train_full.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

não possui nenhum valor ausente.

Outra coisa que devemos fazer é verificar a distribuição dos valores na variável de destino. 

In [19]:
df_train_full.churn.value_counts()

0    4113
1    1521
Name: churn, dtype: int64

Conhecemos os números absolutos, mas vamos verificar também a proporção de usuários churn entre todos os clientes. Para isso, precisamos dividir o número de clientes que desistiram pelo número total de clientes. Sabemos que 1.521 de 5.634 desistiram, então a proporção é

1521 / 5634 = 0.27

Isso nos dá a proporção de usuários cancelados ou a probabilidade de um cliente desistir. Como vemos no conjunto de dados de treinamento, aproximadamente 27% dos clientes pararam de usar nossos serviços e o restante permaneceu como cliente.

In [20]:
# Calculando a taxa de churn com método mean
global_mean = df_train_full.churn.mean()
round(global_mean, 3)

0.27

Nosso conjunto de dados de churn é um exemplo do chamado conjunto de dados `desequilibrado`.Ouve três vezes mais pessoas que não desistiram em nosso conjunto de dados do que aquelas que desistiram.

O oposto de desequilibrado é o caso balanceado , quando as classes positivas e negativas são igualmente distribuídas entre todas as observações .

Ambas as variáveis ​​categóricas e numéricas em nosso conjunto de dados são importantes, mas também são diferentes e precisam de tratamento diferente. Para isso, queremos analisá-los separadamente.

Vamos criar duas listas:

* `categorical`, que conterá os nomes das variáveis ​​categóricas
* `numerical`, que, da mesma forma, terá os nomes das variáveis ​​numéricas

In [21]:
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
               'phoneservice', 'multiplelines', 'internetservice',
               'onlinesecurity', 'onlinebackup', 'deviceprotection',
               'techsupport', 'streamingtv', 'streamingmovies',
               'contract', 'paperlessbilling', 'paymentmethod']
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [22]:
#Primeiro vamos ver quantos valores exclusivos cada variavel possui 
df_train_full[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

Vemos que todas as variáveis têm poucos valores únicos. 

### Importância do Recurso

Saber como outras variáveis ​​afetam a variável de destino, churn, é a chave para entender os dados e construir um bom modelo. Esse processo é chamado de análise de importância de recursos e geralmente é feito como parte da análise exploratória de dados para descobrir quais variáveis ​​serão úteis para o modelo. Ele também nos fornece informações adicionais sobre o conjunto de dados e ajuda a responder perguntas como `“O que faz os clientes desistirem?”` e `“Quais são as características das pessoas que churn ?”`

Temos dois tipos diferentes de características: categóricas e numéricas. Cada tipo tem maneiras diferentes de medir a importância do recurso, então veremos cada um separadamente.

#### **taxa de churn**

Vamos começar examinando as variáveis ​​categóricas. A primeira coisa que podemos fazer é olhar a taxa de churn para cada variável. Sabemos que uma variável categórica tem um conjunto de valores que pode assumir e cada valor define um grupo dentro do conjunto de dados.

Podemos olhar para todos os valores distintos de uma variável. Então, para cada variável, há um grupo de clientes: todos os clientes que têm esse valor. Para cada um desses grupos, podemos calcular a taxa de churn, que é a taxa de churn do grupo. Quando o tivermos, podemos compará-lo com a taxa de churn global – a taxa de churn calculada para todas as observações de uma só vez.

Se a diferença entre as taxas for pequena, o valor não é importante na previsão do churn, pois esse grupo de clientes não é muito diferente do restante dos clientes. Por outro lado, se a diferença não for pequena, algo dentro desse grupo o diferencia dos demais. Um algoritmo de aprendizado de máquina deve ser capaz de pegar isso e usá-lo ao fazer previsões.

In [23]:
# verificando primeiro a variavel gender | fazendo para os clientes do sexo feminino
female_mean = df_train_full[df_train_full.gender == 'female'].churn.mean()
print('gender == female: ', round(female_mean, 3))

# fazendo para os clientes do sexo masculino 
male_mean = df_train_full[df_train_full.gender == 'male'].churn.mean()
print('Gender == Male:', round(male_mean, 3))

gender == female:  0.277
Gender == Male: 0.263


Os números são bem próximos, o que significa que gender não é uma variável útil ao prever a churn

In [24]:
#Vamos verificar a variavel partner (leva valores sim e não)
partner_yes = df_train_full[df_train_full.partner == 'yes'].churn.mean()
print('Partner == Yes: ', round(partner_yes, 3))

partner_no = df_train_full[df_train_full.partner == 'no'].churn.mean()
print('Partner == No: ', round(partner_no, 3))


Partner == Yes:  0.205
Partner == No:  0.33


Como vemos, as taxas para quem tem companheiro são bem diferentes das taxas para quem não tem: 20% e 33%, respectivamente. Isso significa que os clientes sem parceiro têm maior probabilidade de desistir do que aqueles com parceiro

#### Proporção de risco

Além de observar a diferença entre a tarifa do grupo e a tarifa global, é interessante observar a relação entre elas. Em estatística, a razão entre as probabilidades em diferentes grupos é chamada de razão de risco , onde o risco se refere ao risco de ter o efeito. No nosso caso, o efeito é churn, então é o risco de churning :

risco = taxa de grupo / taxa global

Para gender == female, por exemplo, o risco de rotatividade é de 1,02:

risco = 27,7% / 27% = 1,02

Risco é um número entre zero e infinito. Tem uma boa interpretação que informa a probabilidade de os elementos do grupo terem o efeito (churn) em comparação com toda a população.

Se a diferença entre a taxa do grupo e a taxa global for pequena, o risco é próximo de 1: esse grupo tem o mesmo nível de risco que o restante da população. Os clientes do grupo têm tanta probabilidade de rotatividade (churn) quanto qualquer outra pessoa. Em outras palavras, um grupo com risco próximo a 1 não tem risco algum (figura 3.15, grupo A).

Se o risco for menor que 1, o grupo tem riscos menores: o churn neste grupo é menor que o churn global. Por exemplo, o valor 0,5 significa que os clientes deste grupo têm duas vezes menos probabilidade de desistir do que os clientes em geral (figura 3.15, grupo B).

Por outro lado, se o valor for maior que 1, o grupo é arriscado: há mais churn no grupo do que na população. Portanto, um risco de 2 significa que os clientes do grupo têm duas vezes mais chances de desistir

![Texto Alternativo](figure_3-15.png)


In [25]:
# Vamos fazer a proporção de risco para todas as variáveis categóricas 
global_mean = df_train_full.churn.mean() # calcula a media da coluna churn
 
df_group = df_train_full.groupby(by='gender').churn.agg(['mean'])# o df está sendo agrupado com base na coluna gender, a coluna churn é selecionada dentro desse grupo. Por fim, é aplicada a função de agregação mean a essa coluna
df_group['diff'] = df_group['mean'] - global_mean # criamos uma coluna diff onde manteremos a diferença entre a média do grupo e a média global
df_group['risk'] = df_group['mean'] / global_mean # criamos a coluna risk onde calculamos a fração entre a média do grupo e a média global
 
df_group

,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


In [26]:
# Vamos fazer isso para todas as variaveis categóricas. 
# podemos iterar por eles e aplicar o mesmo código para cada um: 
from IPython.display import display 
 
for col in categorical:
    df_group = df_train_full.groupby(by=col).churn.agg(['mean'])
    df_group['diff'] = df_group['mean'] - global_mean
    df_group['rate'] = df_group['mean'] / global_mean
    display(df_group)

,mean,diff,rate
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


,mean,diff,rate
seniorcitizen,,,
0,0.242270,-0.027698,0.897403
1,0.413377,0.143409,1.531208


,mean,diff,rate
partner,,,
no,0.329809,0.059841,1.221659
yes,0.205033,-0.064935,0.759472


,mean,diff,rate
dependents,,,
no,0.313760,0.043792,1.162212
yes,0.165666,-0.104302,0.613651


,mean,diff,rate
phoneservice,,,
no,0.241316,-0.028652,0.893870
yes,0.273049,0.003081,1.011412


...

,mean,diff,rate
streamingtv,,,
no,0.342832,0.072864,1.269897
no_internet_service,0.077805,-0.192163,0.288201
yes,0.302723,0.032755,1.121328


,mean,diff,rate
streamingmovies,,,
no,0.338906,0.068938,1.255358
no_internet_service,0.077805,-0.192163,0.288201
yes,0.307273,0.037305,1.138182


,mean,diff,rate
contract,,,
month-to-month,0.431701,0.161733,1.599082
one_year,0.120573,-0.149395,0.446621
two_year,0.028274,-0.241694,0.104730


,mean,diff,rate
paperlessbilling,,,
no,0.172071,-0.097897,0.637375
yes,0.338151,0.068183,1.252560


,mean,diff,rate
paymentmethod,,,
bank_transfer_(automatic),0.168171,-0.101797,0.622928
credit_card_(automatic),0.164339,-0.105630,0.608733
electronic_check,0.455890,0.185922,1.688682
mailed_check,0.193870,-0.076098,0.718121


A partir dos resultados percebemos que: 

* Quanto ao sexo, não há muita diferença entre mulheres e homens. Ambas as médias são aproximadamente as mesmas, e para ambos os grupos os riscos são próximos de 1.
* Idosos tendem a desistir mais do que não idosos: o risco de rotatividade é de 1,53 para idosos e 0,89 para não idosos.
* Pessoas com um parceiro mudam menos do que pessoas sem parceiro. Os riscos são 0,75 e 1,22, respectivamente.
* As pessoas que usam o serviço telefônico não correm risco de churn: o risco é próximo a 1 e quase não há diferença com a taxa de churn global. As pessoas que não usam o serviço telefônico têm ainda menos probabilidade de churn: o risco é inferior a 1 e a diferença com a taxa de churn global é negativa.

Algumas das variáveis ​​apresentam diferenças bastante significativas:

* Clientes sem suporte técnico tendem a desistir mais do que aqueles que o fazem .
* As pessoas com contratos mensais cancelam o contrato com muito mais frequência do que outras, e as pessoas com contratos de dois anos mudam muito raramente.

![Texto Alternativo](001.jpg)

![Texto Alternativo](002.jpg)

Assim, apenas observando as diferenças e os riscos, podemos identificar as características mais discriminativas: as características que ajudam a detectar o churn

#### Informação mútua

A métrica de informação mútua (mutual information) é um conceito fundamental na teoria da informação e é comumente usado em aprendizado de máquina para avaliar a relação entre duas variáveis. A informação mútua mede a quantidade de informação que as duas variáveis compartilham.

No nosso caso, saber que o cliente tem um contrato mês a mês pode indicar que esse cliente tem mais probabilidade de churn do que não.

Valores mais altos de informação mútua significam maior grau de dependência: se a informação mútua entre uma variável categórica e o alvo for alta, esta variável categórica será bastante útil para prever o alvo. Por outro lado, se a informação mútua for baixa, a variável categórica e o alvo são independentes e, portanto, a variável não será útil para prever o alvo.

A informação mútua já está implementada no Scikit-learn na função mutual_info_score do metricspackage , então podemos apenas usá-lo:

In [27]:
from sklearn.metrics import mutual_info_score
 
def calculate_mi(series): #recebe uma serie de dados como argumento
    return mutual_info_score(series, df_train_full.churn) # retorna o valor da informalção mútua entre a série de dados ea coluna churn
 
df_mi = df_train_full[categorical].apply(calculate_mi) # aplicamos a função calculate_mi a cada coluna dentro da variavel 'categorical'
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI') # O df é classificado em ordem decrescente com base nos valores da informação mutua. to_frame é usado para converter a serie classificada com uma coluna MI 
# a coluna MI representa os valores da informação mútua para cada variavel categórica. 
df_mi

,MI
contract,0.098320
onlinesecurity,0.063085
techsupport,0.061032
internetservice,0.055868
onlinebackup,0.046923
deviceprotection,0.043453
paymentmethod,0.043210
streamingtv,0.031853
streamingmovies,0.031581
paperlessbilling,0.017589


Como vemos, `contract`, `onlinesecurity`, e `techsupport`estão entre as características mais importantes 

#### Coeficiente de correlação

A informação mútua é uma forma de quantificar o grau de dependência entre duas variáveis ​​categóricas, mas não funciona quando uma das características é numérica, por isso não podemos aplicá-la às três variáveis ​​numéricas que temos.

Podemos, no entanto, medir a dependência entre uma variável de destino binária e uma variável numérica. Podemos fingir que a variável binária é numérica (contendo apenas os números zero e um) e então usar os métodos clássicos da estatística para verificar qualquer dependência entre essas variáveis

Coeficiente de correlação (às vezes chamado de coeficiente de correlação de Pearson ). É um valor de –1 a 1:

* Correlação positiva significa que quando uma variável sobe, a outra variável tende a subir também. No caso de um destino binário, quando os valores da variável são altos, vemos uns com mais frequência do que zeros. Mas quando os valores da variável são baixos, os zeros se tornam mais frequentes do que os uns.
* Correlação zero significa nenhuma relação entre duas variáveis: elas são completamente independentes.
* A correlação negativa ocorre quando uma variável sobe e a outra desce. No caso binário, se os valores forem altos, veremos mais zeros do que uns na variável de destino. Quando os valores são baixos, vemos mais uns.

In [28]:
#calculando o coeficiente de correlação para as variaveis numericas
df_train_full[numerical].corrwith(df_train_full.churn)

tenure           -0.351885
monthlycharges    0.196805
totalcharges     -0.196353
dtype: float64

#### verificando os resultados 

* A correlação entre `tenure` e churn é –0,35: tem sinal negativo, então quanto mais tempo os clientes ficam, menos tendem a desistir. Para clientes que permanecem na empresa por dois meses ou menos, o churn rate é de 60%; para clientes com prazo entre 3 e 12 meses, o churn rate é de 40%; e para clientes com permanência superior a um ano, o churn rate é de 17%. Assim, quanto maior o valor da posse, menor a taxa de churn (figura 3.21A) .
* `monthlycharges` tem um coeficiente positivo de 0,19, o que significa que os clientes que pagam mais tendem a sair com mais frequência. Apenas 8% daqueles que pagam menos de US 20 mensais desistiram; clientes que pagam entre US 21 e US 50 rotacionam com mais frequência, com uma taxa de rotatividade de 18%; e 32% das pessoas que pagam mais de US 50 desistiram (figura 3.21B).
* `totalcharges` tem uma correlação negativa, o que faz sentido: quanto mais tempo as pessoas ficam na empresa, mais elas pagam no total, então é menos provável que elas saiam. Neste caso, esperamos um padrão semelhante ao `tenure`. Para valores pequenos, a taxa de churn é alta; para valores maiores, é menor.

## Engenharia de Recursos 

**Codificação one-hot para variáveis categóricas**

Precisamos converter nossos dados categóricos em uma forma de matriz ou codificar. 

In [29]:
# convertendo nosso df em uma lista de dicionários
train_dict = df_train[categorical + numerical].to_dict(orient='records')

Cada coluna do dataframe é a chave neste dicionário, com valores provenientes dos valores reais da linha do dataframe.

Agora podemos usar DictVectorizer. Nós o criamos e depois o encaixamos na lista de dicionários que criamos anteriormente:

In [30]:
from sklearn.feature_extraction import DictVectorizer
 
dv = DictVectorizer(sparse=False) #sparse=False significa que a matriz não será esparsa, em vez disso, será uma matriz numpy simples
dv.fit(train_dict)

DictVectorizer(sparse=False)

Neste código criamos um DictVectorizerexemplo, que chamamos dv, e "treiná-lo" invocando o fitmétodo . O fitO método analisa o conteúdo desses dicionários e descobre os valores possíveis para cada variável e como mapeá-los para as colunas na matriz de saída. Se um recurso for categórico, ele aplicará o esquema de codificação one-hot, mas se for numérico, ele permanecerá intacto.

Depois de ajustar o vetorizador, podemos usá-lo para converter os dicionários em uma matriz usando o transformmétodo :

In [31]:
X_train = dv.transform(train_dict) # Depois de ajustar o vetorizador, podemos usá-lo para converter os dicionários em uma matriz usando o transformmétodo :

In [32]:
# vamos dar uma olhada na primeira linha
X_train[0]

array([0.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00,
       1.0000e+00, 0.0000e+00, 0.0000e+00, 8.6100e+01, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 7.1000e+01, 6.0459e+03])

Podemos aprender os nomes de todas as colunas: 

In [34]:
dv.get_feature_names()

['contract=month-to-month',
 'contract=one_year',
 'contract=two_year',
 'dependents=no',
 'dependents=yes',
 'deviceprotection=no',
 'deviceprotection=no_internet_service',
 'deviceprotection=yes',
 'gender=female',
 'gender=male',
 'internetservice=dsl',
 'internetservice=fiber_optic',
 'internetservice=no',
 'monthlycharges',
 'multiplelines=no',
 'multiplelines=no_phone_service',
 'multiplelines=yes',
 'onlinebackup=no',
 'onlinebackup=no_internet_service',
 'onlinebackup=yes',
 'onlinesecurity=no',
 'onlinesecurity=no_internet_service',
 'onlinesecurity=yes',
 'paperlessbilling=no',
 'paperlessbilling=yes',
 'partner=no',
 'partner=yes',
 'paymentmethod=bank_transfer_(automatic)',
 'paymentmethod=credit_card_(automatic)',
 'paymentmethod=electronic_check',
 'paymentmethod=mailed_check',
 'phoneservice=no',
 'phoneservice=yes',
 'seniorcitizen',
 'streamingmovies=no',
 'streamingmovies=no_internet_service',
 'streamingmovies=yes',
 'streamingtv=no',
 'streamingtv=no_internet_servic

Como vemos, para cada recurso categórico, ele cria várias colunas para cada um de seus valores distintos. Para `contract`, Nós temos `contract=month-to-month`, `contract=one_year`, e `contract=two_year`, e para `dependents`, Nós temos `dependents=no` e `dependents=yes`. Recursos como `tenure` e `totalcharges` mantem os nomes originais porque são numéricos; portanto, `DictVectorizer` não os muda .

## Treinamento 

**Regressão Logística** 


A regressão logística é um modelo estatístico amplamente utilizado para prever a probabilidade de ocorrência de um evento binário, como sucesso ou fracasso, sim ou não, verdadeiro ou falso. É uma técnica de aprendizado supervisionado que se baseia na análise da relação entre variáveis independentes e uma variável dependente categórica.


**Função Sigmoide** 

Precisamos da função sigmoide na regressão logística porque ela mapeia a saída do modelo para um intervalo de valores entre 0 e 1, representando uma probabilidade. Isso permite interpretar a saída do modelo como a probabilidade de um evento binário ocorrer. Além disso, a função sigmoide possui uma forma curva que é adequada para modelar relações não lineares entre as variáveis independentes e a variável dependente.

Resumidamente a Regressão Logística nada mais é do que uma Regressão Linear com a função sigmoide para garantir que as previsões do modelo sempre fique entre 0 e 1. 

In [35]:
# função sigmoide
import math 

def sigmoid(score): 
    return 1 / (1 + math.exp(-score))

In [36]:
# regressão logistica para python 
def logistic_regression(xi):
    score = bias
    for j in range(n):
        score = score + xi[j] * w[j]
    prob = sigmoid(score)
    return prob

Os parâmetros do modelo de regressão logística são os mesmos da regressão linear:

w0 = é o termo de viés (bias). 
w = ( w1, w2, ..., wn ) = é o vetor de pesos. 

In [37]:
# importando o modelo 
from sklearn.linear_model import LogisticRegression 

# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [39]:
# Treinamos o modelo chamando o método fit
model = LogisticRegression(solver='liblinear', random_state=1) #solver default atualmente é lbfgs, O liblinear é uma boa escolha para datasets pequenos.
model.fit(X_train, y_train)

LogisticRegression(random_state=1, solver='liblinear')

Parâmetros do modelo: 

`solver` - Refere-se ao algoritmo usado para otimizar a função de perda na regressão logística. Ele determina como o modelo encontra os melhores coeficientes para ajustar os dados.

`random_state` - Ele define a semente para a geração dos números aleatórios usados no treinamento do modelo.

### Aplicando aos dados de validação 

Precisamos aplicar primeiramente a Codificação one-hot a todas as variáveis categórias no conjunto de dados de validação. 

In [41]:
# Primeiro, converter o dataframe em uma lista de dicionarios
val_dict = df_val[categorical + numerical].to_dict(orient='records')

# Em seguida alimentamos o DictVectorizer 
X_val = dv.transform(val_dict)

# X_val agora é uma matriz com recursos do conjunto de dados de validação

Para obter as probabilidades, usamos o método `predict_ proba` do modelo:

In [43]:
y_pred = model.predict_proba(X_val)


O resultado de `predict_proba` é uma matriz NumPy bidimensional ou uma matriz de duas colunas. A primeira coluna da matriz contém a probabilidade de que o alvo seja negativo (o cliente não vai desistir) e a segunda coluna contém a probabilidade de que o alvo seja positivo (o alvo é um e o cliente irá desistir)

In [45]:
# Não precisamos das duas colunas, precisamos apenas da segunda 
y_pred = model.predict_proba(X_val)[:, 1]

Como resultado, obtemos uma matriz NumPy unidimensional que contém apenas os valores da segunda coluna.

Lembre-se de como queríamos usar esse modelo: queríamos fidelizar os clientes, identificando aqueles que estão prestes a rescindir o contrato com a empresa e enviando mensagens promocionais, oferecendo descontos e outros benefícios. Fazemos isso na esperança de que, após receberem o benefício, permaneçam na empresa. Por outro lado, não queremos dar promoções a todos os nossos clientes, porque isso nos prejudicará financeiramente: teremos menos lucro, se houver.

Para tomar a decisão real de enviar uma carta promocional aos nossos clientes, usar apenas a probabilidade não é suficiente. Precisamos de concretas previsões - valores binários de True(churn, então envie o e-mail) ou False(não churn, então não envie o e-mail).

Para obter as previsões binárias, pegamos as probabilidades e as cortamos acima de um certo limite. Se a probabilidade de um cliente for maior que esse limite, prevemos o churn, caso contrário, não a churn. Se selecionarmos 0,5 como esse limite, é fácil fazer as previsões binárias. Nós apenas usamos o “ >=” operador:

In [46]:
y_pred >= 0.5

array([False, False, False, ..., False,  True, False])

In [47]:
# anexando os resultados em uma variavel churn 
churn = y_pred >= 0.5

### Avaliação Simples 

Calculando a precisão: vamos pegar cada previsão e compará-la com o valor real. 

In [49]:
(y_val == churn).mean()

0.8016129032258065

Isso significa que nosso modelo faz as previsões corretas em 80% dos casos. 

### Interpretação do modelo

In [50]:
# obtendo o termo de viés 
model.intercept_[0]

-0.12198856763403255

In [51]:
# obtendo o resto dos pesos 
model.coef_[0]

array([ 5.63350752e-01, -8.59117734e-02, -5.99427547e-01, -3.02788796e-02,
       -9.17096881e-02,  9.99295858e-02, -1.15869847e-01, -1.06048306e-01,
       -2.73675348e-02, -9.46210328e-02, -3.23344998e-01,  3.17226277e-01,
       -1.15869847e-01,  7.84241261e-04, -1.68098703e-01,  1.27132848e-01,
       -8.10227123e-02,  1.35702916e-01, -1.15869847e-01, -1.41821636e-01,
        2.57855957e-01, -1.15869847e-01, -2.63974677e-01, -2.12616055e-01,
        9.06274878e-02, -4.80222880e-02, -7.39662796e-02, -2.66755924e-02,
       -1.36243628e-01,  1.74743460e-01, -1.33812807e-01,  1.27132848e-01,
       -2.49121416e-01,  2.97088104e-01, -8.48581097e-02, -1.15869847e-01,
        7.87393891e-02, -9.90741031e-02, -1.15869847e-01,  9.29553825e-02,
        1.78137876e-01, -1.15869847e-01, -1.84256597e-01, -6.94867149e-02,
        4.47687718e-04])

Para ver qual recurso está associado a cada peso, vamos usar o método`get_feature_names`

In [52]:
dict(zip(dv.get_feature_names(), model.coef_[0].round(3)))

C:\Users\55119\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


{'contract=month-to-month': 0.563,
 'contract=one_year': -0.086,
 'contract=two_year': -0.599,
 'dependents=no': -0.03,
 'dependents=yes': -0.092,
 'deviceprotection=no': 0.1,
 'deviceprotection=no_internet_service': -0.116,
 'deviceprotection=yes': -0.106,
 'gender=female': -0.027,
 'gender=male': -0.095,
 'internetservice=dsl': -0.323,
 'internetservice=fiber_optic': 0.317,
 'internetservice=no': -0.116,
 'monthlycharges': 0.001,
 'multiplelines=no': -0.168,
 'multiplelines=no_phone_service': 0.127,
 'multiplelines=yes': -0.081,
 'onlinebackup=no': 0.136,
 'onlinebackup=no_internet_service': -0.116,
 'onlinebackup=yes': -0.142,
 'onlinesecurity=no': 0.258,
 'onlinesecurity=no_internet_service': -0.116,
 'onlinesecurity=yes': -0.264,
 'paperlessbilling=no': -0.213,
 'paperlessbilling=yes': 0.091,
 'partner=no': -0.048,
 'partner=yes': -0.074,
 'paymentmethod=bank_transfer_(automatic)': -0.027,
 'paymentmethod=credit_card_(automatic)': -0.136,
 'paymentmethod=electronic_check': 0.175,


###  Usando o modelo

Primeiro, pegamos um cliente que queremos pontuar e colocamos todos os valores das variáveis ​​em um dicionário:



In [53]:
customer = {
    'customerid': '8879-zkjof',
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'no',
    'dependents': 'no',
    'tenure': 41,
    'phoneservice': 'yes',
    'multiplelines': 'no',
    'internetservice': 'dsl',
    'onlinesecurity': 'yes',
    'onlinebackup': 'no',
    'deviceprotection': 'yes',
    'techsupport': 'yes',
    'streamingtv': 'yes',
    'streamingmovies': 'yes',
    'contract': 'one_year',
    'paperlessbilling': 'yes',
    'paymentmethod': 'bank_transfer_(automatic)',
    'monthlycharges': 79.85,
    'totalcharges': 3320.75,
}

**OBSERVAÇÃO** 

Quando preparamos itens para previsão, eles devem passar pelas mesmas etapas de pré-processamento que fizemos para treinar o modelo. Se não fizermos exatamente da mesma maneira, o modelo pode não obter as coisas que espera ver e, neste caso, as previsões podem realmente falhar. É por isso que no exemplo anterior, no customerdicionário, os nomes dos campos e os valores das strings ficam em minúsculas e os espaços são substituídos por sublinhados.

Vamos usar nosso modelo agora para ver se esse cliente vai ou não desistir. 

In [54]:
#convertendo o dicionário em uma matriz 
X_test = dv.transform([customer])

In [55]:
X_test

array([[0.00000e+00, 1.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 1.00000e+00, 1.00000e+00, 0.00000e+00,
        1.00000e+00, 0.00000e+00, 0.00000e+00, 7.98500e+01, 1.00000e+00,
        0.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00, 1.00000e+00,
        1.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00,
        0.00000e+00, 0.00000e+00, 1.00000e+00, 4.10000e+01, 3.32075e+03]])

Vemos vários recursos de codificação únicos (uns e zeros), bem como alguns numéricos ( monthlycharges, tenure, e totalcharges).

Agora pegamos esta matriz e a colocamos no modelo treinado: 

In [57]:
model.predict_proba(X_test)

array([[0.92667755, 0.07332245]])

Ele returna uma matriz com as previsões. 
Para cada cliente ele gera dois numeros, que são a probabilidade de permanecer na empresa e a probabilidade churn(não permanecer) 

In [58]:
# filtrando apenas para pegar o valor da primeira linha e da segunda coluna
model.predict_proba(X_test)[0, 1]

0.0733224506719823

O resultado é a a probabilidade de churn desse cliente

0,073 - A probabilidade desse cliente desistir é de 7% 

**Testando com outro cliente**

In [59]:
customer = {
    'gender': 'female',
    'seniorcitizen': 1,
    'partner': 'no',
    'dependents': 'no',
    'phoneservice': 'yes',
    'multiplelines': 'yes',
    'internetservice': 'fiber_optic',
    'onlinesecurity': 'no',
    'onlinebackup': 'no',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'yes',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 85.7,
    'totalcharges': 85.7
}

In [60]:
# fazendo a previsão 
X_test = dv.transform([customer])
model.predict_proba(X_test)[0, 1]

0.8321647912113965

A saída do modelo é de 83% de probabilidade de churn, portanto, devemos enviar a esse cliente um e-mail promocional na esperança de retê-lo .